In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#pip install plotly

In [ ]:
df = pd.read_csv('../input/covid19-case-surveillance-public-use-dataset/COVID-19_Case_Surveillance_Public_Use_Data.csv')

**The purpose of this project is to build a ML algorithm to define whether or not the person with the features from the dataset will be hospitalized.**

In [ ]:
#df.head()
#df.shape
#df.isnull().sum()
#df['current_status'].unique()

In [ ]:
df.describe()

In [ ]:
import plotly.express as px

fig = px.histogram(df['age_group'])
fig.show()

In [ ]:
grouped_df = df.groupby(['sex','age_group','Race and ethnicity (combined)','hosp_yn','medcond_yn']).count()
df1 = pd.DataFrame()
df1['sex'] = np.array(list(grouped_df.index))[:,0]
df1['age_group'] = np.array(list(grouped_df.index))[:,1]
df1['race'] = np.array(list(grouped_df.index))[:,2]
df1['hosp'] = np.array(list(grouped_df.index))[:,3]
df1['medc'] = np.array(list(grouped_df.index))[:,4]
df1['Count'] = grouped_df['icu_yn'].values

In [ ]:
fig = px.sunburst(df1, path=['hosp','medc','age_group','race','sex'], values='Count', color='hosp')
fig.show()

In [ ]:
del df['cdc_report_dt']
del df['pos_spec_dt']
del df['onset_dt']


df.drop(df[df['current_status'] == 'Probable Case'].index, inplace = True)
df.drop(df[df['hosp_yn'] == 'Missing'].index, inplace = True)
df.drop(df[df['Race and ethnicity (combined)'] == 'Unknown'].index, inplace = True)

del df['current_status']

In [ ]:
df = df.dropna()

In [ ]:
df1 = df[:1000]
df1

In [ ]:
from sklearn import preprocessing 

label_encoder = preprocessing.LabelEncoder() 

df1 = df1.apply(label_encoder.fit_transform)

In [ ]:
df.head()

In [ ]:
y = df1['hosp_yn']
X = df1[['sex','age_group','Race and ethnicity (combined)','medcond_yn']]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

# Let's begin exploring the algorithms!

We start with SVM.

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [ ]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

CV_svm = GridSearchCV(SVC(), tuned_parameters)
CV_svm.fit(X_train, y_train)
CV_svm.best_params_

In [ ]:
svm = SVC(C=1.0, kernel='linear', gamma=0.001)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

print(accuracy_score(y_test, y_pred_svm))
print(confusion_matrix(y_test, y_pred_svm))

Now let's create a pipeline for a standard procedure for other algorithms to use.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

tree_para = {'criterion':['gini','entropy'],
             'max_depth':[1,2,3,5,10,50,100]}
CV_tree = GridSearchCV(DecisionTreeClassifier(), tree_para, cv=5)
CV_tree.fit(X_train, y_train)
CV_tree.best_params_

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=2, criterion='gini')
tree.fit(X_train, y_train)

y_pred_tree = tree.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_pred_tree))
print(confusion_matrix(y_test, y_pred_tree))

Let's build an ExtraTrees classifier to get feature importances of the datase

In [ ]:
import matplotlib.pyplot as plt

from sklearn.ensemble import ExtraTreesClassifier

# Build a classification task using 3 informative features

# Build a forest and compute the impurity-based feature importances
forest = ExtraTreesClassifier(n_estimators=4,
                              random_state=0)

forest.fit(X_train, y_train)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X_train.shape[1]), importances[indices],
        color="b", yerr=std[indices], align="center")
plt.xticks(range(X_train.shape[1]), indices)
plt.xlim([-1, X_train.shape[1]])
plt.show()

In [ ]:
X_train, y_train

**Looks like the only important value is whether or not the patient has a medical condition.**